In [1]:
# data_path_train_orig = '/content/drive/MyDrive/boosters_semrush/data/semrush_cup_train_data.csv'
# data_path_test_orig = '/content/drive/MyDrive/boosters_semrush/data/semrush_cup_test_data.csv'
# data_path_sample_orig = '/content/drive/MyDrive/boosters_semrush/data/semrush_cup_sub.csv'
# data_path_dict_orig = '/content/drive/MyDrive/boosters_semrush/data/semrush_cup_categories_and_demo.csv'

# cache_path = '/content/drive/MyDrive/boosters_semrush/cache'

In [2]:
data_path_train_orig = 'data/semrush_cup_train_data.csv'
data_path_test_orig = 'data/semrush_cup_test_data.csv'
data_path_sample_orig = 'data/semrush_cup_sub.csv'
data_path_dict_orig = 'data/semrush_cup_categories_and_demo.csv'

cache_path = 'cache'

In [3]:
!nvidia-smi

/usr/bin/sh: 1: nvidia-smi: not found


# Init

In [4]:
# !pip3 install sklearn
# !pip install tensorflow 
# !pip3 install tensorflow_addons

In [5]:
import pandas as pd
import pickle
import os
from pathlib import Path

from sklearn.model_selection import train_test_split

In [6]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers

2022-04-13 11:34:32.664461: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-13 11:34:32.664492: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
def read_csv_cached(csv_path, cache_path):
    data = pd.read_csv(csv_path)
    return data 

#     print(f'read_csv_cached: csv_path={csv_path} cache_path={cache_path}')

#     os.makedirs(cache_path, exist_ok=True)

#     csv_path_cached_pickle = Path(cache_path) / (Path(csv_path).name + '.pickle')
#     print(f'read_csv_cached: csv_path_cached_pickle={csv_path_cached_pickle}')

#     if not(Path(csv_path_cached_pickle).exists()):
#         print(f'read_csv_cached: missing cache')
#         data = pd.read_csv(csv_path)
        
#         with open(csv_path_cached_pickle, 'wb') as f:
#             pickle.dump(data, f)
#     else:
#         print(f'read_csv_cached: cached')

#     print(f'read_csv_cached: reading from cache')
#     with open(csv_path_cached_pickle, 'rb') as f:      
#         data = pickle.load(f)
    
#     print(f'read_csv_cached: done')
#     return data

def split_train(train_path, cache_path):
    print(f'split_train: train_path={train_path} cache_path={cache_path}')

    os.makedirs(cache_path, exist_ok=True)

    data_path_cached_pickle = Path(cache_path) / ('split_train' + '.pickle')
    print(f'split_train: data_path_cached_pickle={data_path_cached_pickle}')

    if not(Path(data_path_cached_pickle).exists()):
        print(f'split_train: missing cache')

        train = read_csv_cached(train_path, cache_path)

        uids_list = train['device_id'].unique()
        train_train_uids, train_test_uids = train_test_split(uids_list, train_size=0.6, random_state=7)
        train_test_uids, train_valid_uids = train_test_split(train_test_uids, train_size=0.5, random_state=7)

        train_train_data = train.loc[train['device_id'].isin(train_train_uids)]
        train_test_data = train.loc[train['device_id'].isin(train_test_uids)]
        train_valid_data = train.loc[train['device_id'].isin(train_valid_uids)]

        data = (train_train_data, train_test_data, train_valid_data)
        
        with open(data_path_cached_pickle, 'wb') as f:
            pickle.dump(data, f)
    else:
        print(f'split_train: cached')

    print(f'split_train: reading from cache')
    with open(data_path_cached_pickle, 'rb') as f:      
        data = pickle.load(f)
    
    train_train, train_test, train_valid = data
    print(f'data size: train={len(train_train)}, test={len(train_test)}, valid={len(train_valid)}')

    return data

# Read data

In [8]:
train_train, train_test, train_valid = split_train(data_path_train_orig, cache_path)

split_train: train_path=data/semrush_cup_train_data.csv cache_path=cache
split_train: data_path_cached_pickle=cache/split_train.pickle
split_train: cached
split_train: reading from cache
data size: train=7810396, test=2632773, valid=2597474


In [9]:
train_train[train_train.event_group_id==39147]

,event_group_id,device_id,timestamp,domain,referrer_domain,url,referrer_url,referrer_num,is_referrer
0,39147,100000,1174857389,ttGJTM.AK,ttNggB.jg,tWAmxZ.ttGJTM.AK/jCYBFH/jlYGps/jjGbCS/jrsypg,tWAmxZ.ttNggB.jg/jdSNxb,0,False
1,39147,100000,1174857499,ttNggB.jg,NaN,tWAmxZ.ttNggB.jg/jHSjLY,NaN,1,False
2,39147,100000,1174857514,ttdgZf.jg,ttNggB.jg,ttdgZf.jg/jdSNxb,tWAmxZ.ttNggB.jg/jdSNxb,2,False
3,39147,100000,1174857523,ttNggB.jg,ttdgZf.jg,tWAmxZ.ttNggB.jg/jHSjLY,ttdgZf.jg/jdSNxb,3,False
4,39147,100000,1174857529,ttsUFm.MZ,ttNggB.jg,ttsUFm.MZ/jjdsdQ/jHiiZc/jHICvz/jjIijp/jdLZBy/j...,tWAmxZ.ttNggB.jg/jdSNxb,4,False
5,39147,100000,1174857546,ttBWMn.jg,ttNggB.jg,tWAmxZ.ttBWMn.jg/jHLvKC/jdSNxb,tWAmxZ.ttNggB.jg/jdSNxb,5,True
6,39147,100000,1174857562,tWxuXo.MZ,ttNggB.jg,ttjvjg.tWxuXo.MZ/jHKZdc/jrbQKv/jjHNIG/jHgCli/j...,tWAmxZ.ttNggB.jg/jdSNxb,6,False
7,39147,100000,1174857665,ttxgCt.jg,ttBWMn.jg,tWAmxZ.ttxgCt.jg/jjxlps,tWAmxZ.ttBWMn.jg/jjGuNS/jjcbKW,7,False
8,39147,100000,1174857670,tWzTqL.jg,ttxgCt.jg,ttuNso.tWzTqL.jg/jvjYYW,tWAmxZ.ttxgCt.jg/jjxlps,8,False
9,39147,100000,1174857671,ttkgWu.jg,tWzTqL.jg,tWAmxZ.ttkgWu.jg/jHcdBZ/jdSNxb,ttuNso.tWzTqL.jg/jvjYYW,9,False


In [10]:
# sample = read_csv_cached(data_path_sample_orig, cache_path)

In [11]:
# sample

# Load Domains

In [12]:
print('Loading domains from train.csv')
domains1 = read_csv_cached(data_path_train_orig, cache_path)
domains1 = domains1[['domain']]
domains1['domain'] = domains1['domain'].fillna("EMPTY")
domains1 = domains1.drop_duplicates()
print(f'domains1: {len(domains1)}')

print('Loading domains from test.csv')
domains2 = read_csv_cached(data_path_test_orig, cache_path)
domains2 = domains2[['domain']]
domains2['domain'] = domains2['domain'].fillna("EMPTY")
domains2 = domains2.drop_duplicates()
print(f'domains2: {len(domains2)}')

domains = pd.concat([domains1, domains2])
domains = domains.drop_duplicates()
# domains = domains.sort_values()
domains = domains.reset_index(drop=True)
print(f'domains: {len(domains)}')
del (domains1, domains2)

Loading domains from train.csv
domains1: 947918
Loading domains from test.csv
domains2: 569788
domains: 1212771


In [13]:
domains[['domain1', 'domain2', 'domain3', 'domain4']] = domains.domain.apply(lambda x: x[::-1]).str.split(".", expand=True).fillna('EMPTY')

domains = domains.set_index('domain')
for i in range(1,5):
    print(i)
    domains[f'domain{i}'] = domains.groupby(f'domain{i}').ngroup()
    domains = domains.rename(columns={f"domain{i}": f"domain{i}_id"})

domains['domain_id'] = domains.groupby('domain').ngroup()

1
2
3
4


In [14]:
domains.head()

,domain1_id,domain2_id,domain3_id,domain4_id,domain_id
domain,,,,,
ttGJTM.AK,58,263721,10272,46,410567
ttNggB.jg,191,51044,10272,46,566993
ttdgZf.jg,191,584688,10272,46,797192
ttsUFm.MZ,161,698170,10272,46,1050429
ttBWMn.jg,191,734745,10272,46,331063


In [15]:
domains_dict_id =  {index:value for index, value in domains.domain_id.items()}
domains1_dict_id = {index:value for index, value in domains.domain1_id.items()}
domains2_dict_id = {index:value for index, value in domains.domain2_id.items()}
domains3_dict_id = {index:value for index, value in domains.domain3_id.items()}
domains4_dict_id = {index:value for index, value in domains.domain4_id.items()}
# domains_dict_id

In [16]:
# domains = domains.union(set(train.domain.fillna("EMPTY").unique()))

In [17]:
# domains = set()

# train = read_csv_cached(data_path_train_orig, cache_path)
# domains = domains.union(set(train.domain.fillna("EMPTY").unique()))
# del(train)

# test = read_csv_cached(data_path_test_orig, cache_path)
# domains = domains.union(set(test.domain.fillna("EMPTY").unique()))
# del(test)

# print(len(domains))

In [18]:
# # domains = train_train.domain.unique()
# domains = list(set(np.concatenate([train_train.domain.fillna("EMPTY").unique(),
#                                    train_train.referrer_domain.fillna("EMPTY").unique(),
#                                    train_test.domain.fillna("EMPTY").unique(),
#                                    train_test.referrer_domain.fillna("EMPTY").unique(),
#                                    train_valid.domain.fillna("EMPTY").unique(),
#                                    train_valid.referrer_domain.fillna("EMPTY").unique(),                                   
#                                    ])))
# print(len(domains))
# domains_dict_id = {key:value for value, key in enumerate(domains)}

# Process data

In [19]:
def preprocess_data(data_raw):
    print('Processing history')
    data1 = data_raw[data_raw.referrer_num!='target'].copy()

    data1_domain = data1.domain.fillna('EMPTY').copy()
    data1['domain1'] = data1_domain.map(domains1_dict_id).astype('int')
    data1['domain2'] = data1_domain.map(domains2_dict_id).astype('int')
    data1['domain3'] = data1_domain.map(domains3_dict_id).astype('int')
    data1['domain4'] = data1_domain.map(domains4_dict_id).astype('int')    
    data1['domain']  = data1_domain.map(domains_dict_id).astype('int')

    # data1 = data1[['event_group_id', 'timestamp', 'domain', 'is_referrer', 'referrer_num']]

    print('. Pivoting history')
    data1 = data1.pivot(index='event_group_id', columns='referrer_num', values=['timestamp', 'domain', 'is_referrer', 'domain1', 'domain2', 'domain3', 'domain4'])

    print('. Updating pivot dtypes')
    data1.iloc[:,:70] = data1.iloc[:,:70].astype('int')
    # print(data1.columns)

    print('Processing target')
    data2 = data_raw[data_raw.referrer_num == 'target']
    data2 = data2[['event_group_id', 'timestamp', 'domain']]

    data2_domain = data2.domain.fillna('EMPTY').copy()
    data2['domain1'] = data2_domain.map(domains1_dict_id).astype('int')
    data2['domain2'] = data2_domain.map(domains2_dict_id).astype('int')
    data2['domain3'] = data2_domain.map(domains3_dict_id).astype('int')
    data2['domain4'] = data2_domain.map(domains4_dict_id).astype('int')
    data2['domain']  = data2_domain.map(domains_dict_id).astype('int')

    data2 = data2.set_index('event_group_id')    
    # print(data2.columns)

    print('. Merging results')
    data_new = data1.join(data2)
    data_new.iloc[:,:10] = np.repeat(np.expand_dims(data_new['timestamp'].values, axis=1), 10, axis=1) - data_new.iloc[:,:10].values
    data_new = data_new.drop(columns='timestamp')
    # print(data_new.columns)

    return data_new

In [20]:
train_train = preprocess_data(train_train)
train_test = preprocess_data(train_test)
# train_valid = preprocess_data(train_valid)

Processing history
. Pivoting history
. Updating pivot dtypes
Processing target
. Merging results


/home/studio-lab-user/.conda/envs/d2l/lib/python3.9/site-packages/pandas/core/frame.py:9130: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


Processing history
. Pivoting history
. Updating pivot dtypes
Processing target
. Merging results


# Get Dataset

In [21]:
def get_raw_dataset(data):
    timeframe_history = tf.data.Dataset.from_tensor_slices(np.expand_dims(data.iloc[:,:10].values, axis=1))
    domain_history = tf.data.Dataset.from_tensor_slices(np.expand_dims(data.iloc[:,10:20].values, axis=1))

    domain1_history = tf.data.Dataset.from_tensor_slices(np.expand_dims(data.iloc[:,30:40].values, axis=1))
    domain2_history = tf.data.Dataset.from_tensor_slices(np.expand_dims(data.iloc[:,40:50].values, axis=1))
    domain3_history = tf.data.Dataset.from_tensor_slices(np.expand_dims(data.iloc[:,50:60].values, axis=1))
    domain4_history = tf.data.Dataset.from_tensor_slices(np.expand_dims(data.iloc[:,60:70].values, axis=1))

    domain = tf.data.Dataset.from_tensor_slices(data.domain.values)    
    labels = tf.data.Dataset.from_tensor_slices(data.iloc[:,20:30].values)

    features_dataset = tf.data.Dataset.zip((timeframe_history, domain_history, 
                                            domain1_history, domain2_history, domain3_history, domain4_history,
                                            domain))
    dataset = tf.data.Dataset.zip((features_dataset, labels))

    return dataset

def get_train_dataset(data):
    dataset = get_raw_dataset(data)
    dataset = dataset.shuffle(64*20)    
    dataset = dataset.batch(64)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)   

    return dataset

def get_test_dataset(data):
    dataset = get_raw_dataset(data)
    dataset = dataset.batch(64)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

train_train_dataset = get_train_dataset(train_train)
train_test_dataset = get_test_dataset(train_test)

for i in get_raw_dataset(train_train).take(1):
    print(i)

2022-04-13 11:38:13.608933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-13 11:38:13.608967: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-13 11:38:13.608994: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2022-04-13 11:38:13.613905: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


((<tf.Tensor: shape=(1, 10), dtype=int64, numpy=array([[118, 117, 117, 116, 110, 105, 101,  98,  79,   2]])>, <tf.Tensor: shape=(1, 10), dtype=int64, numpy=
array([[830860, 234725, 562463,  69133, 234725, 706371, 561235, 549981,
        566993, 346652]])>, <tf.Tensor: shape=(1, 10), dtype=int64, numpy=array([[191, 191, 191, 191, 191, 191, 191, 191, 191, 210]])>, <tf.Tensor: shape=(1, 10), dtype=int64, numpy=
array([[151945, 588159, 908767, 314255, 588159, 741681, 170296, 217123,
         51044, 306111]])>, <tf.Tensor: shape=(1, 10), dtype=int64, numpy=
array([[10272, 10272, 10272, 10272, 10272, 10272, 10272, 10272, 10272,
         1180]])>, <tf.Tensor: shape=(1, 10), dtype=int64, numpy=array([[46, 46, 46, 46, 46, 46, 46, 46, 46, 46]])>, <tf.Tensor: shape=(), dtype=int64, numpy=1168608>), <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])>)


# Create model

In [22]:
input_timestamp_history = tf.keras.layers.Input(shape=(1,10), dtype=tf.int32, name='timestamp_history')
timestamp_history = tf.cast(input_timestamp_history, tf.float32)  / 1000
timestamp_history = tf.expand_dims(timestamp_history, 3)

domain_embedding = tf.keras.layers.Embedding(input_dim=len(domains_dict_id), output_dim=8)
domain1_embedding = tf.keras.layers.Embedding(input_dim=len(domains1_dict_id), output_dim=8)
domain2_embedding = tf.keras.layers.Embedding(input_dim=len(domains2_dict_id), output_dim=8)
domain3_embedding = tf.keras.layers.Embedding(input_dim=len(domains3_dict_id), output_dim=8)
domain4_embedding = tf.keras.layers.Embedding(input_dim=len(domains4_dict_id), output_dim=8)


input_domain_history = tf.keras.layers.Input(shape=(1,10), dtype=tf.int32, name='domain_history')
domain_history = domain_embedding(input_domain_history)

input_domain1_history = tf.keras.layers.Input(shape=(1,10), dtype=tf.int32, name='domain1_history')
domain1_history = domain1_embedding(input_domain1_history)

input_domain2_history = tf.keras.layers.Input(shape=(1,10), dtype=tf.int32, name='domain2_history')
domain2_history = domain2_embedding(input_domain2_history)

input_domain3_history = tf.keras.layers.Input(shape=(1,10), dtype=tf.int32, name='domain3_history')
domain3_history = domain3_embedding(input_domain3_history)

input_domain4_history = tf.keras.layers.Input(shape=(1,10), dtype=tf.int32, name='domain4_history')
domain4_history = domain4_embedding(input_domain4_history)

# input_referrer_domain_history = tf.keras.layers.Input(shape=(1,10), dtype=tf.int32, name='referrer_domain_history')
# referrer_domain_history = domain_embedding(input_referrer_domain_history)

history = tf.keras.layers.Concatenate()([timestamp_history, domain_history])
history = tf.squeeze(history, 1)
history = layers.SpatialDropout1D(0.20)(history)

history = tf.keras.layers.GRU(256, return_sequences=True)(history)
history = tf.keras.layers.GRU(128, return_sequences=False)(history)

##
# history_max = tf.keras.layers.GlobalMaxPooling1D()(history)
# history_avg = tf.keras.layers.GlobalAveragePooling1D()(history)
# history = tf.keras.layers.Concatenate()([history_max, history_avg])
##

input_domain = tf.keras.layers.Input(shape=(), dtype=tf.int32, name='domain')
domain = domain_embedding(input_domain)
output = tf.keras.layers.Concatenate()([history, domain])

output = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.L1L2(1e-7, 1e-5))(output)
output = tf.keras.layers.Dense(10, activation='softmax')(output)

model = tf.keras.Model(inputs=(input_timestamp_history, input_domain_history,
                               input_domain1_history, input_domain2_history, input_domain3_history, input_domain4_history,
                               input_domain), outputs=output)
model.summary()

model.predict(train_train_dataset.take(2))

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 timestamp_history (InputLayer)  [(None, 1, 10)]     0           []                               
                                                                                                  
 tf.cast (TFOpLambda)           (None, 1, 10)        0           ['timestamp_history[0][0]']      
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 1, 10)        0           ['tf.cast[0][0]']                
                                                                                                  
 domain_history (InputLayer)    [(None, 1, 10)]      0           []                               
                                                                                              

array([[0.0991885 , 0.10079766, 0.10089838, ..., 0.09884706, 0.10079177,
        0.10191014],
       [0.09975296, 0.10237639, 0.09938625, ..., 0.09942356, 0.09849713,
        0.10250196],
       [0.10008019, 0.10149033, 0.0998697 , ..., 0.09854023, 0.10003991,
        0.10090977],
       ...,
       [0.09956571, 0.10067805, 0.09967744, ..., 0.10072471, 0.10010724,
        0.1012039 ],
       [0.09926265, 0.1016297 , 0.10039549, ..., 0.0988146 , 0.09992678,
        0.10201303],
       [0.09941099, 0.09969334, 0.10044973, ..., 0.09963947, 0.10026305,
        0.1002931 ]], dtype=float32)

In [23]:
model.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=tfa.metrics.F1Score(num_classes=10, average='macro')
)

# Fit the model

In [24]:
train_history = model.fit(train_train_dataset,
                          validation_data=train_test_dataset,
                          epochs=5)

Epoch 1/5
11095/11095 [==============================] - 1611s 145ms/step - loss: 1.1843 - f1_score: 0.3844 - val_loss: 0.9963 - val_f1_score: 0.4768
Epoch 2/5
11095/11095 [==============================] - 1614s 146ms/step - loss: 0.8667 - f1_score: 0.5439 - val_loss: 0.9677 - val_f1_score: 0.5025
Epoch 3/5
11095/11095 [==============================] - 1626s 147ms/step - loss: 0.7125 - f1_score: 0.6144 - val_loss: 1.0082 - val_f1_score: 0.5066
Epoch 4/5
11095/11095 [==============================] - 1643s 148ms/step - loss: 0.6192 - f1_score: 0.6591 - val_loss: 1.0657 - val_f1_score: 0.4998
Epoch 5/5
  951/11095 [=>............................] - ETA: 24:29 - loss: 0.5767 - f1_score: 0.6705

KeyboardInterrupt: 

# Validate on train_test

In [ ]:
train_test_pred = model.predict(train_test_dataset, verbose=1)

In [ ]:
train_test_fact = train_test.iloc[:,20:30].values

In [ ]:
from sklearn.metrics import f1_score  

In [ ]:
th_list = np.arange(0.1, 0.51, 0.05)
f1_list = []
for th in th_list:
    f1_list.append(f1_score(train_test_fact, train_test_pred >= th, average='macro'))

pd.DataFrame({'th':th_list, 'f1':f1_list}).sort_values('f1', ascending=False)

In [ ]:
train_test_pred_max = np.zeros_like(train_test_pred)
arg_max_result = np.argmax(train_test_pred, axis=1)
for i,j in enumerate(arg_max_result):
    train_test_pred_max[i][j] = 1
f1_score(train_test_fact, train_test_pred_max, average='macro')

In [ ]:
train_test_fact_v2 = np.argmax(train_test_fact, axis=1)
train_test_fact_v2

train_test_pred_v2 = np.argmax(train_test_pred, axis=1)
train_test_pred_v2

f1_score(train_test_fact_v2, train_test_pred_v2, average='macro')

# Validate on train_valid

In [ ]:
train_valid_dataset = get_test_dataset(train_valid)
train_valid_pred = model.predict(train_valid_dataset, verbose=1)
train_valid_fact = train_valid.iloc[:,20:30].values

In [ ]:
th_list = np.arange(0.1, 0.51, 0.05)
f1_list = []
for th in th_list:
    f1_list.append(f1_score(train_valid_fact, train_valid_pred >= th, average='macro'))

pd.DataFrame({'th':th_list, 'f1':f1_list}).sort_values('f1', ascending=False)

# Prediction

In [ ]:
test = read_csv_cached(data_path_test_orig, cache_path)

In [ ]:
test = preprocess_data(test, domains_dict_id)